In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [2]:
def load_and_preprocess_images(folder_path, image_size=(64, 64)):
    image_data = []
    labels = []

    for label_idx, label_name in enumerate(os.listdir(folder_path)):
        label_folder = os.path.join(folder_path, label_name)
        
        for image_filename in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_filename)
            image = Image.open(image_path).convert('L')  # Convert to grayscale
            image = image.resize(image_size)
            image = np.array(image) / 255.0  # Normalize pixel values
            image_data.append(image)
            labels.append(label_idx)

    return np.array(image_data), np.array(labels)

In [3]:
dataset_train = "Dataset/train"
dataset_test = "Dataset/test"

In [4]:
X_train, Y_train = load_and_preprocess_images(dataset_train)
X_test, Y_test = load_and_preprocess_images(dataset_test)

In [8]:
import cv2
import numpy as np

# Function to perform data augmentation on a single image
def augment_image(image):
    # Randomly rotate the image by a small angle (-15 to +15 degrees)
    angle = np.random.randint(-15, 16)
    rows, cols = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

    # Randomly flip the image horizontally (50% chance)
    if np.random.rand() > 0.5:
        image = np.fliplr(image)

    # Randomly adjust the brightness of the image
    brightness_factor = 0.5 + np.random.rand()  # Range: [0.5, 1.5]
    image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)

    return image

# Augment the images in X_train and update y_train
augmented_X_train = []
augmented_y_train = []  # Initialize an empty list for the labels
for image, label in zip(X_train, Y_train):
    augmented_image = augment_image(image)
    augmented_X_train.append(augmented_image)
    augmented_y_train.append(label)

augmented_X_train = np.array(augmented_X_train)
augmented_y_train = np.array(augmented_y_train)


X_train = np.concatenate((X_train, augmented_X_train), axis=0)
Y_train = np.concatenate((Y_train, augmented_y_train), axis=0)


In [9]:
input_size = X_train.shape[1] * X_train.shape[2]
hidden_size = 64
output_size = 3
learning_rate = 0.01
num_epochs = 2000

In [10]:
np.random.seed(42)
weights_input_hidden = np.random.randn(input_size, hidden_size)
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size)
bias_output = np.zeros((1, output_size))

In [11]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [12]:
for epoch in range(num_epochs):
    # Forward pass
    hidden_layer_input = np.dot(X_train.reshape(-1, input_size), weights_input_hidden) + bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    output_layer_output = sigmoid(output_layer_input)

    # Compute loss (cross-entropy)
    num_examples = len(X_train)
    y_encoded = np.zeros((num_examples, output_size))
    y_encoded[range(num_examples), Y_train] = 1
    loss = -np.sum(y_encoded * np.log(output_layer_output + 1e-9)) / num_examples

    # Backpropagation
    d_loss = (output_layer_output - y_encoded) / num_examples
    d_output_layer = d_loss * output_layer_output * (1 - output_layer_output)
    d_hidden_layer = np.dot(d_output_layer, weights_hidden_output.T) * hidden_layer_output * (1 - hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, d_output_layer)
    bias_output -= learning_rate * np.sum(d_output_layer, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * np.dot(X_train.reshape(-1, input_size).T, d_hidden_layer)
    bias_hidden -= learning_rate * np.sum(d_hidden_layer, axis=0, keepdims=True)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


Epoch 0, Loss: 0.7351
Epoch 100, Loss: 0.8157
Epoch 200, Loss: 0.8568
Epoch 300, Loss: 0.8764
Epoch 400, Loss: 0.8846
Epoch 500, Loss: 0.8858
Epoch 600, Loss: 0.8824
Epoch 700, Loss: 0.8753
Epoch 800, Loss: 0.8651
Epoch 900, Loss: 0.8518
Epoch 1000, Loss: 0.8357
Epoch 1100, Loss: 0.8175
Epoch 1200, Loss: 0.7983
Epoch 1300, Loss: 0.7795
Epoch 1400, Loss: 0.7620
Epoch 1500, Loss: 0.7458
Epoch 1600, Loss: 0.7312
Epoch 1700, Loss: 0.7185
Epoch 1800, Loss: 0.7078
Epoch 1900, Loss: 0.6988


In [13]:
hidden_layer_input = np.dot(X_test.reshape(-1, input_size), weights_input_hidden) + bias_hidden
hidden_layer_output = sigmoid(hidden_layer_input)
output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
output_layer_output = sigmoid(output_layer_input)


In [14]:
predictions = np.argmax(output_layer_output, axis=1)


In [15]:
accuracy = accuracy_score(Y_test, predictions)
f1_m = f1_score(Y_test, predictions, average='macro')
f1_w = f1_score(Y_test, predictions, average='weighted')

In [16]:
print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Macro F1 Score: {f1_m:.4f}")
print(f"Weighted F1 Score: {f1_w:.4f}")

Test Accuracy: 62.82%
Macro F1 Score: 0.4127
Weighted F1 Score: 0.5016
